# Python test for machine-learning

### Importing data

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR
import seaborn as sns
import io
import urllib.request
import matplotlib.pyplot as plt
import os
import gzip
import collections
import re
import json
import xml.etree.ElementTree as ET
import zipfile

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Databases to include
* ChEmbl
* [DrugDatabank](https://www.drugbank.ca)
* [Kegg](https://www.genome.jp/kegg/drug/)
* [UnitProtKB](https://www.uniprot.org/help/uniprotkb)
* [PubChem](https://pubchem.ncbi.nlm.nih.gov/)
* [Drug Product Database](https://health-products.canada.ca/api/documentation/dpd-documentation-en.html#a1)
* [PharmGKB](https://www.pharmgkb.org/)
* [Therapeutic Targets Database](http://bidd.nus.edu.sg/group/cjttd/)
* [ChemSpider](http://www.chemspider.com/)
* [ChEbi](https://www.ebi.ac.uk/chebi/)
* ZINC

chEmbl information was obtained through a select of the database (into mysql in centOS)

In [ ]:
data=pd.read_csv('product_adme.csv')
data.head()

For Drugbank Import, download from the website the compressed file, see [here](https://www.drugbank.ca/releases/latest)  
The gzip format works best, so if in *.zip* format, convert to *.gz*

In [ ]:
#for drug databank
def drugbank_import(path, create_alias='N'):
    
    with gzip.open(path) as xml_file:
        tree = ET.parse(xml_file)
    root = tree.getroot()

    ns = '{http://www.drugbank.ca}'
    calc = "{ns}calculated-properties/{ns}property"
    exp = "{ns}experimental-properties/{ns}property"
    extern = "{ns}external-identifiers/{ns}external-identifier"
    inchikey_template = calc+"[{ns}kind='InChIKey']/{ns}value"
    inchi_template = calc+"[{ns}kind='InChI']/{ns}value"

    melt_point_template = exp+"[{ns}kind='Melting Point']/{ns}value"
    Hydrophobicity_template = exp+"[{ns}kind='Hydrophobicity']/{ns}value"
    isoelectric_template = exp+"[{ns}kind='Isoelectric Point']/{ns}value"
    molweight_template = exp+"[{ns}kind='Molecular Weight']/{ns}value"
    molform_template = exp+"[{ns}kind='Molecular Formula']/{ns}value"
    logP_template = exp+"[{ns}kind='logP']/{ns}value"
    logS_template = exp+"[{ns}kind='logS']/{ns}value"
    boil_template = exp+"[{ns}kind='Boiling Point']/{ns}value"
    caco_template = exp+"[{ns}kind='caco2 Permeability']/{ns}value"
    water_exp_template = exp+"[{ns}kind='Water Solubility']/{ns}value"
    pKa_template = exp+"[{ns}kind='pKa']/{ns}value"


    psa_template = calc+"[{ns}kind='Polar Surface Area (PSA)']/{ns}value"
    refr_template = calc+"[{ns}kind='Refractivity']/{ns}value"
    pola_template = calc+"[{ns}kind='Polarizability']/{ns}value"
    bioa_template = calc+"[{ns}kind='Bioavailability']/{ns}value"
    ghose_template = calc+"[{ns}kind='Ghose Filter']/{ns}value"
    mddr_template = calc+"[{ns}kind='MDDR-Like Rule']/{ns}value"

    # external identifiers
    DPD_template = extern + \
        "[{ns}resource='Drugs Product Database (DPD)']/{ns}identifier"
    PubChem_template = extern+"[{ns}resource='PubChem Substance']/{ns}identifier"
    kegg_template = extern+"[{ns}resource='KEGG Drug']/{ns}identifier"
    GKB_template = extern+"[{ns}resource='PharmGKB']/{ns}identifier"
    UPKB_template = extern+"[{ns}resource='UniProtKB']/{ns}identifier"
    TTD_template = extern + \
        "[{ns}resource='Therapeutic Targets Database']/{ns}identifier"
    wiki_template = extern+"[{ns}resource='Wikipedia']/{ns}identifier"
    ChEMBL_template = extern+"[{ns}resource='ChEMBL']/{ns}identifier"

    rows = list()
    for i, drug in enumerate(root):
        row = collections.OrderedDict()
        assert drug.tag == ns + 'drug'
        row['type'] = drug.get('type')
        row['drugbank_id'] = drug.findtext(ns + "drugbank-id[@primary='true']")
        row['average-mass'] = drug.findtext(ns + "average-mass")
        row['monoisotopic-mass'] = drug.findtext(ns + "monoisotopic-mass")
        row['name'] = drug.findtext(ns + "name")
        # free text
        row['volume-of-distribution'] = drug.findtext(
            ns + "volume-of-distribution")
        row['clearance'] = drug.findtext(ns + "clearance")
        row['half-life'] = drug.findtext(ns + "half-life")
        row['toxicity'] = drug.findtext(ns + "toxicity")
        row['metabolism'] = drug.findtext(ns + "metabolism")
        row['absorption'] = drug.findtext(ns + "absorption")
        row['smiles'] = drug.findtext(ns + "smiles")
        # experimental
        row['melting point'] = drug.findtext(melt_point_template.format(ns=ns))
        row['Hydrophobicity'] = drug.findtext(
            Hydrophobicity_template.format(ns=ns))
        row['Isoelectric Point'] = drug.findtext(
            isoelectric_template.format(ns=ns))
        row['Molecular Weight'] = drug.findtext(molweight_template.format(ns=ns))
        row['Molecular Formula'] = drug.findtext(molform_template.format(ns=ns))
        row['logP EXP'] = drug.findtext(logS_template.format(ns=ns))
        row['logS EXP'] = drug.findtext(melt_point_template.format(ns=ns))
        row['pKa EXP'] = drug.findtext(pKa_template.format(ns=ns))
        row['Boiling Point'] = drug.findtext(boil_template.format(ns=ns))
        row['Caco2 Permeability'] = drug.findtext(caco_template.format(ns=ns))
        row['Water Solubility EXP'] = drug.findtext(
            water_exp_template.format(ns=ns))
        # calculated
        row['PSA calc'] = drug.findtext(psa_template.format(ns=ns))
        row['Refractivity calc'] = drug.findtext(refr_template.format(ns=ns))
        row['Polarizability'] = drug.findtext(pola_template.format(ns=ns))
        row['Bioavailability'] = drug.findtext(bioa_template.format(ns=ns))
        row['Ghose Filter'] = drug.findtext(ghose_template.format(ns=ns))
        row['MDDR-Like Rule'] = drug.findtext(mddr_template.format(ns=ns))
        # external
        row['Drugs Product Database (DPD)'] = drug.findtext(
            DPD_template.format(ns=ns))
        row['PubChem Substance'] = drug.findtext(PubChem_template.format(ns=ns))
        row['KEGG Drug'] = drug.findtext(kegg_template.format(ns=ns))
        row['PharmGKB'] = drug.findtext(GKB_template.format(ns=ns))
        row['UniProtKB'] = drug.findtext(UPKB_template.format(ns=ns))
        row['Therapeutic Targets Database'] = drug.findtext(
            TTD_template.format(ns=ns))
        row['Wikipedia'] = drug.findtext(wiki_template.format(ns=ns))
        row['ChEMBL'] = drug.findtext(ChEMBL_template.format(ns=ns))

        # others
        row['groups'] = [group.text for group in
                         drug.findall("{ns}groups/{ns}group".format(ns=ns))]
        row['atc_codes'] = [code.get('code') for code in
                            drug.findall("{ns}atc-codes/{ns}atc-code".format(ns=ns))]
        row['categories'] = [x.findtext(ns + 'category') for x in
                             drug.findall("{ns}categories/{ns}category".format(ns=ns))]
        row['inchi'] = drug.findtext(inchi_template.format(ns=ns))
        row['inchikey'] = drug.findtext(inchikey_template.format(ns=ns))
        
        # Add drug aliases
        aliases = {
            elem.text for elem in
            drug.findall("{ns}international-brands/{ns}international-brand".format(ns=ns)) +
            drug.findall("{ns}synonyms/{ns}synonym[@language='English']".format(ns=ns)) +
            drug.findall("{ns}international-brands/{ns}international-brand".format(ns=ns)) +
            drug.findall("{ns}products/{ns}product/{ns}name".format(ns=ns))

        }
        aliases.add(row['name'])
        row['aliases'] = sorted(aliases)

        rows.append(row)
        
    if create_alias=='Y':
        alias_dict = {row['drugbank_id']: row['aliases'] for row in rows}
        with open('aliases.json', 'w') as fp:
            json.dump(alias_dict, fp, indent=2, sort_keys=True)
        


    def collapse_list_values(row):
        for key, value in row.items():
            if isinstance(value, list):
                row[key] = '|'.join(value)
        return row


    rows = list(map(collapse_list_values, rows))

    columns = ['drugbank_id', 'name', 'type', 'groups', 'atc_codes',
               'categories', 'inchikey', 'inchi', 'average-mass',
               'monoisotopic-mass', 'volume-of-distribution', 'clearance', 'half-life',
               'toxicity', 'metabolism', 'metabolism', 'absorption', 'smiles',
               'melting point', 'logS EXP', 'logP EXP', 'pKa EXP', 'Isoelectric Point', 'Molecular Weight', 'Molecular Formula',
               'Hydrophobicity', 'Boiling Point', 'Caco2 Permeability', 'Water Solubility EXP', 'PSA calc', 'Refractivity calc', 'Polarizability', 'Ghose Filter', 'MDDR-Like Rule',
               'Drugs Product Database (DPD)', 'PubChem Substance', 'KEGG Drug', 'PharmGKB', 'UniProtKB', 'Therapeutic Targets Database', 'ChEMBL', 'Wikipedia']

    drugbank_df = pd.DataFrame.from_dict(rows)[columns]

    drugbank_slim_df = drugbank_df[
        drugbank_df.groups.map(lambda x: 'approved' in x) &
        drugbank_df.inchi.map(lambda x: x is not None) &
        drugbank_df.type.map(lambda x: x == 'small molecule')
    ]

    # write drugbank tsv
    #drugbank_df.to_csv('drugbank.csv', sep=";", index=False)

    #drugbank_slim_df.to_csv('drugbank-slim.csv', sep=";", index=False)
    return drugbank_df,drugbank_slim_df

In [ ]:
dbdata,dbdataslim=drugbank_import('../Drugbank/full database.xml.gz')
#dbdata,dbdataslim=drugbank_import('drugbank_all_full_database.xml.zip')

dbdata.head()

to merge the two sources of data

In [ ]:
full=pd.merge(data, dbdata, left_on='chembl_id', right_on='ChEMBL', how='outer', suffixes=('_left', '_right'))
full.head()

In [ ]:
full.dtypes
data=full

**Evaluating the dataframe created** 
* evaluating datatypes
* evaluating certain columns 
* filtering data
* histogram for numerical values

In [ ]:
#1
data.dtypes
#2
data['published_type'].describe()
#3
data[data["published_type"] == "logP"]
#4
data.hist(column="standard_value")

### para filtrar por count é:

In [ ]:
def evaluate_column(df,column,filter=0):
    if df[column].dtypes=='object':
        eval=df.groupby(column).filter(lambda x: len(x) > filter)
        #plot
        eval[column].value_counts().plot(kind='bar')
        #print list
        eval[column].value_counts().index
        
        info=[#df[column].sum(), # Total sum of the column values
             #df[column].mean(), # Mean of the column values
             #df[column].median(), # Median of the column values
             df[column].nunique(), # Number of unique entries
             #df[column].max(), # Maximum of the column values
             #df[column].min(), # Minimum of the column values
             df[column].isnull().sum(),
        df[column].describe()] 

    else:
        eval=df.hist(column=column)
        info=[df[column].sum(), # Total sum of the column values
             df[column].mean(), # Mean of the column values
             df[column].median(), # Median of the column values
             df[column].nunique(), # Number of unique entries
             df[column].max(), # Maximum of the column values
             df[column].min(), # Minimum of the column values
             df[column].isnull().sum(),
             df[column].describe()] 
        
    return info
evaluate_column(data,'Boiling Point',25)

In [ ]:
#teste purposes
data_eval=data[data["published_type"] == "pKa"]
[len(data_eval)*0.8,
 data_eval['acd_most_apka'].sum(), # Total sum of the column values
 data_eval['acd_most_apka'].mean(), # Mean of the column values
 data_eval['acd_most_apka'].median(), # Median of the column values
 data_eval['acd_most_apka'].nunique(), # Number of unique entries
 data_eval['acd_most_apka'].max(), # Maximum of the column values
 data_eval['acd_most_apka'].min(),
 data_eval['acd_logd'].isna().sum()] # Minimum of the column values

## Cleaning data
#### filtering for pka
pka_data_cleaning from [here](https://www.kaggle.com/mnoori/feature-selection-for-mlr-with-python)


In [ ]:
def data_cleaning(df,null_cutoff,published_type,remove_null='Y'):
    
    #removing duplicate and unmeaninfull columns
    df=df.drop(columns=["md", "cp","cr","at","aa","molregno.2","molregno.1","doc_id.1","doc_id.2","molregno.3","doc_id.1","doc_id.2"])
    df=df.drop(columns=["src_id.1","chembl_id.1","assay_id.1","record_id.1"])
    
    #selecting for pka
    df=df[df["published_type"] == published_type]

    #keeping interesting columns
    df=df[["max_phase","dosed_ingredient", "structure_type",  "molecule_type",
    "oral", "parenteral", "topical", "black_box_warning",
    "natural_product", "first_in_class", "chirality", "prodrug",
    "inorganic_flag", "usan_year", "availability_type", "usan_stem",
    "polymer_flag", "usan_substem", "usan_stem_definition",
    "indication_class", "withdrawn_flag", "withdrawn_year",
    "withdrawn_country", "withdrawn_reason", "mw_freebase","alogp","hba",
    "hbd", "psa", "rtb", "ro3_pass", "num_ro5_violations", "acd_most_apka",
    "acd_most_bpka", "acd_logp", "acd_logd", "molecular_species",
    "full_mwt", "aromatic_rings", "heavy_atoms", "num_alerts",
    "qed_weighted", "mw_monoisotopic",  "hba_lipinski",
    "hbd_lipinski", "num_lipinski_ro5_violations","assay_type", "relationship_type",
    "confidence_score","standard_value","half-life","toxicity","metabolism","absorption",
    "smiles","melting point","logS EXP","logP EXP","pKa EXP","Isoelectric Point",
    "Molecular Weight","Molecular Formula","Hydrophobicity","Boiling Point","Caco2 Permeability",
    "Water Solubility EXP","PSA calc","Refractivity calc","Polarizability","Ghose Filter","MDDR-Like Rule",
    "average-mass","monoisotopic-mass", "volume-of-distribution","clearance"]]
    

    #removing outlier far greater than average
    if published_type in ["pKa"]:
        df=df[df["standard_value"]<400]
    
    if remove_null=='N':
        null_cutoff=0
        
    #dropping columns with more than a missing values
    #null_values=df.isnull().sum()
    #drop_missing_values=null_values[null_values>(null_cutoff*len(df))]
    #df=df.drop(drop_missing_values.index, axis=1)    
    df = df.dropna(thresh=len(df)-null_cutoff*len(df), axis=1)
    
    # counting null values in text columns
    text_cols_nullcount=df.select_dtypes(include=["object"]).isnull().sum().sort_values(ascending=False)
    text_cols_nullcols=text_cols_nullcount.index
    for col in text_cols_nullcols:
        mostcounts=df[col].value_counts().index.tolist()
        df[col]=df[col].fillna(mostcounts[0]) #replacing the missing column in a text with the highest number of values

    #missing values in numerical columns 
    num_cols=df.select_dtypes(include=["object","float64"]).columns #selecting numerical columns
    num_null_counts=df[num_cols].isnull().sum().sort_values(ascending=False) #counting null values in columns
    num_null_cols=num_null_counts[num_null_counts!=0].index #selecting the ones that have missing values
    df=df.fillna(df[num_null_cols].mode().to_dict(orient="records")[0]) #replacing missing with mode
    #passing categorical to numerical
    df=pd.get_dummies(df, prefix="is_")#add column to name as well

    #remove duplicates
    df=df.drop_duplicates()
    
    return df

In [ ]:
pka_data=data_cleaning(data,0.8,'pKa',remove_null='N')
pka_data

In [ ]:
pka_data.dtypes

In [ ]:
logp_data=data_cleaning(data,0.8,'logP')
logp_data

analisado o gráfico em baixo, vemos que não existe correlação forte de de nenhuma coluna com o alvo - **standard_value**

In [ ]:
logd_data=data_cleaning(df=data,published_type='logD',null_cutoff=0.8)    
logd_data

In [ ]:
def check_correlation(df,target,corr_cutoff):
    data_train=df.sample(frac=0.7,random_state=200)
    data_test=df.drop(data_train.index)

    data_x=df.drop(columns=[target])
    data_y=df[target]

    data_x_train=data_train.drop(columns=[target])
    data_y_train=data_train[target]

    data_x_test=data_test.drop(columns=[target])
    data_y_test=data_test[target]
    
    corr=data_train.corr()
    #fig,ax=plt.subplots(figsize=(8,6))
    #sns.heatmap(corr)
    features=''
    features_text=''
    if len(corr[target].where(lambda x : x.abs()>corr_cutoff).dropna())>1:
        features=corr[target].where(lambda x : x.abs()>corr_cutoff).dropna()
        features_text=features.index.str.cat(sep=', ')+'\n'
    else:
        features='1'
        features_text='None'
    #print('The features correlated with target above the threshold %s are %s' %(corr_cutoff,features_text))
    return len(features)

check_correlation(pka_data,'standard_value',0.01)

para retirar colunas com variância abaixo de X, mas devolve um np  
além disso, temos variaveis "booleanas" o que torna complicado aplicar isto pq a variância não há de ser muito grande

In [ ]:
#sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
#sel.fit_transform(pka_data_corrected_1)

In [ ]:
def xMlr(df,target,frac=0.7,cv=10):
    i=0
    mse=0
    score=0
    while i<cv:
        np.random.seed(seed=123)
        pka_data_train=df.sample(frac=0.7,random_state=200)
        pka_data_test=df.drop(pka_data_train.index)

        pka_data_x=df.drop(columns=[target])
        pka_data_y=df[target]

        pka_data_x_train=df.drop(columns=[target])
        pka_data_y_train=df[target]

        pka_data_x_test=df.drop(columns=[target])
        pka_data_y_test=df[target]
        regr = linear_model.LinearRegression()
        regr.fit(pka_data_x_train, pka_data_y_train)
        #print(regr.coef_)
        mse+=(np.mean((regr.predict( pka_data_x_test)-pka_data_y_test)**2))
        score+=regr.score(pka_data_x_test, pka_data_y_test)
        i+=1
    return mse/cv, score/cv


MSE is mean squared error  
Explained variance score:   
1 is perfect prediction
and 0 means that there is no linear relationship
between X and y.

In [ ]:
mlr_mse,mlr_score=xMlr(pka_data,'standard_value')
print("RMSE is %s. Score is %s." % (mlr_mse, mlr_score))

# SVR

In [ ]:
def xSVR(df,target,frac=0.7,cv=10):
    i=0
    mse=0
    score=0
    while i<cv:
        np.random.seed(seed=123)
        pka_data_train=df.sample(frac=0.7,random_state=200)
        pka_data_test=df.drop(pka_data_train.index)

        pka_data_x=df.drop(columns=[target])
        pka_data_y=df[target]

        pka_data_x_train=df.drop(columns=[target])
        pka_data_y_train=df[target]

        pka_data_x_test=df.drop(columns=[target])
        pka_data_y_test=df[target]
        clf = SVR(gamma='scale', C=1.0, epsilon=0.1)
        clf.fit(pka_data_x_train, pka_data_y_train) 
        mse+=(np.mean((clf.predict( pka_data_x_test)-pka_data_y_test)**2))
        score+=clf.score(pka_data_x_test, pka_data_y_test, sample_weight=None)
        i+=1
    return mse/cv, score/cv

In [ ]:
svr_mse,svr_score=xSVR(pka_data,'standard_value')
print("MSE is %s. Score is %s." % (svr_mse, svr_score))

In [ ]:
def evaluation_test_train(df,length,cv,null_cutoff,correlation):
    result={}
    eval_df=df.groupby("published_type").filter(lambda x: len(x) > length)
    test_list=eval_df["published_type"].value_counts().index
    
    for item in test_list:
        try:
            test_df=data_cleaning(df,null_cutoff,str(item))
            if check_correlation(corr_cutoff=correlation,df=test_df,target='standard_value')>1:
                result[item]=xMlr(test_df,'standard_value',frac=0.7,cv=cv)
        except:
            print(item)
            continue
    return result

data_result=evaluation_test_train(data,1000,5,0.8,0.1)
print (data_result)